### Example notebook to use prospect to inspect specific spectra

In the context of minisv2, March 2020

In [1]:
# Load modules :
# - use DESI master kernel
# - get prospect from desihub : https://github.com/desihub/prospect/tree/minisv
#     use the minisv branch (as of early March)
#     setup pythonpath, eg :
import sys, os
sys.path.append("/global/homes/a/armengau/prospect/py")
from prospect import plotframes,utils_specviewer,myspecselect
import desispec.io
from astropy.table import Table, vstack

In [2]:
# Prospect's main function usage
help(plotframes.plotspectra)

Help on function plotspectra in module prospect.plotframes:

plotspectra(spectra, nspec=None, startspec=None, zcatalog=None, model_from_zcat=True, model=None, notebook=False, is_coadded=True, title=None, html_dir=None, with_imaging=True, with_noise=True, with_coaddcam=True, mask_type='DESI_TARGET', with_thumb_tab=True, with_vi_widgets=True, with_thumb_only_page=False)
    Main prospect routine. From a set of spectra, creates a bokeh document used for VI, to be displayed as an HTML page or within a jupyter notebook.
    
    Main Parameter
    ---------
    spectra: input spectra. Supported formats: 1) a 3-band DESI spectra object, with bands 'b', 'r', 'z'. 2) a single-band 
        DESI spectra object, bandname 'brz'. 2) a list of 3 frames, associated to the b, r and z bands.
    zcatalog (default None): astropy Table, containing the 'ZBEST' output redrock. Currently supports only redrock-PCA files.
        The entries in zcatalog are matched to spectra with TARGETID, so that each spec

In [3]:
# 1) Prepare spectra
# This is just an example, using Anand's custom coadd files, and making some random selection
coadd_dir="/global/cscratch1/sd/raichoor/desi_spectro/70002/00052961-00052962-00053122-00053123-00053124-00053125"
coadd_names=[x for x in os.listdir(coadd_dir) if "coadd" in x and ".fits" in x]

all_spec = None
for thecoadd in coadd_names :
    spec = desispec.io.read_spectra(coadd_dir+"/"+thecoadd)
    spec.fibermap['EXPID'] = spec.fibermap['FIBER']*0 # TMP TRICK (??) seems expid is required for Spectra.update()
    if all_spec is None : all_spec = spec
    else : all_spec.update(spec)
spectra = utils_specviewer.specviewer_selection(all_spec,mask='SV0_QSO', mask_type='CMX_TARGET', gmag_cut=[19,20])
print(str(spectra.num_spectra())+" spectra selected")

22 spectra selected


In [5]:
# 1bis) Prepare redrock table
# Optional step (prospect can run with spectra alone)
# The table must contain all TargetIDs of selected spectra, but a one-to-one match is not needed
zbest_names = [x.replace('coadd','zbest') for x in coadd_names]
ztables = []
for zz in zbest_names : 
    ztables.append(Table.read(coadd_dir+"/"+zz,'ZBEST'))
zcat = vstack(ztables)
print(str(len(zcat))+" entries in z catalog")

2500 entries in z catalog


In [ ]:
# 2) Show the viewer
# - Use a reasonable amount of spectra to display (few tens..) to limit the amount of data inside the notebook
# - Don't forget to specify the mask_type (default is DESI_TARGET)
# - Documentation on using the viewer itself is somewhere else...
plotframes.plotspectra(spectra, zcatalog=zcat, title='QSOS - 19<g<20', notebook=True, mask_type='CMX_TARGET')